Supervised preprocessing and visualization

In [2]:
# Importiere benötigte Bibliotheken
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder

In [3]:
#Lade den Datensatz
df = pd.read_csv("C:\\Users\\sonja\\Downloads\\diabetes.csv")

In [14]:
#Funktion zur Erkennung von Ausreißern basierend auf dem IQR
def detect_outliers_iqr(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    return (df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))


In [21]:

# Konvertiere alle Spalten in numerische Werte, ersetze fehlerhafte Werte mit NaN
df = df.apply(pd.to_numeric, errors='coerce')

In [22]:
# Funktion zur Erkennung von Ausreißern mit IQR
def detect_outliers_iqr(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    return (df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))


In [23]:
# Ersetze Ausreißer mit NaN
df_outliers = detect_outliers_iqr(df)
df[df_outliers] = np.nan


In [24]:
# Entferne Zeilen mit NaN-Werten
df_clean = df.dropna()


In [25]:
print("Bereinigter Datensatz:")
print(df_clean.head())

Bereinigter Datensatz:
Empty DataFrame
Columns: [preg, plas, pres, skin, insu, mass, pedi, age, class]
Index: []


In [29]:
# Feature ranking using Chi-squared test
X = df_clean.drop(columns=['class'])  # Features
y = df_clean['class']  # Target


In [30]:
# Normalize features for chi2 test
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

chi_scores, p_values = chi2(X_scaled, y)
feature_ranking = pd.DataFrame({
    'Feature': X.columns,
    'Chi2 Score': chi_scores
}).sort_values(by='Chi2 Score', ascending=False)


NameError: name 'MinMaxScaler' is not defined